In [ ]:
! pip install torch-geometric

In [ ]:
# load train dataset
train_dataset = LRGBDataset(root='data/LRGBDataset', name='Peptides-struct', split='train')

In [ ]:
# Node-level clustering
## apply gaussian mixture models on train dataset
import torch
from sklearn.mixture import GaussianMixture
from torch_geometric.nn.pool import max_pool

cluster_graphs = []
labels_list = []
for data in train_dataset:
    n_clusters = 3 
    gmm = GaussianMixture(n_components=n_clusters, random_state=42)
    
    node_features = data.x.detach().cpu().numpy() # convert to numpy array    
    gmm_labels = gmm.fit_predict(node_features)
    tensor_labels = torch.from_numpy(gmm_labels).long() # convert to tensor
    
    # Ensure edge_index and cluster labels are contiguous
    tensor_labels = tensor_labels.contiguous()
    data.edge_index = data.edge_index.contiguous()
    
    # Apply max pooling
    cluster_graph = max_pool(tensor_labels, data, transform=None)
    
    # Store results
    cluster_graphs.append(cluster_graph)
    labels_list.append(tensor_labels)